# Corrección de Inconsistencias - Telecom X

## Propósito
En este cuaderno, se corrigen las inconsistencias identificadas en el dataset de Telecom X, con un enfoque particular en la columna `Churn`, que contiene valores vacíos. También se verifica la coherencia de otras variables categóricas y numéricas para preparar los datos para el análisis exploratorio.

## Tarea
1. Cargar los datos desde la URL de la API y desanidar las columnas JSON.
2. Corregir los valores vacíos en la columna `Churn`.
3. Verificar la coherencia de variables categóricas y numéricas.
4. Guardar el dataset corregido para su uso posterior.

In [2]:
# Importación de bibliotecas
import pandas as pd
import requests
import json

# Carga de datos desde la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()

# Conversión a DataFrame
df = pd.DataFrame(data)

# Desanidar columnas JSON
customer_df = pd.json_normalize(df['customer'])
customer_df.columns = [f"customer_{col}" for col in customer_df.columns]
phone_df = pd.json_normalize(df['phone'])
phone_df.columns = [f"phone_{col}" for col in phone_df.columns]
internet_df = pd.json_normalize(df['internet'])
internet_df.columns = [f"internet_{col}" for col in internet_df.columns]
account_df = pd.json_normalize(df['account'])
account_df.columns = [f"account_{col}" for col in account_df.columns]

# Combinar los datos desanidados
df_clean = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# 1. Corrección de la columna 'Churn'
print("Valores únicos en 'Churn' antes de la corrección:")
print(df_clean['Churn'].value_counts())

# Reemplazar valores vacíos ('') por 'No'
df_clean['Churn'] = df_clean['Churn'].replace('', 'No')

# Verificar corrección
print("\nValores únicos en 'Churn' después de la corrección:")
print(df_clean['Churn'].value_counts())

# 2. Corrección de tipos de datos para variables numéricas
# Forzar conversión de 'account_Charges.Total' y 'account_Charges.Monthly' a float64
df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')
df_clean['customer_tenure'] = df_clean['customer_tenure'].astype('int64')
df_clean['customer_SeniorCitizen'] = df_clean['customer_SeniorCitizen'].astype('int64')

# 3. Verificación de coherencia en variables categóricas
categorical_columns = [
    'customer_gender', 'customer_Partner', 'customer_Dependents',
    'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService',
    'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
    'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies',
    'account_Contract', 'account_PaperlessBilling', 'account_PaymentMethod'
]
print("\nValores únicos en variables categóricas:")
for col in categorical_columns:
    print(f"\n{col}:")
    print(df_clean[col].value_counts())

# 4. Verificación de coherencia en variables numéricas
numeric_columns = ['customer_tenure', 'account_Charges.Monthly', 'account_Charges.Total']
print("\nEstadísticas descriptivas de variables numéricas:")
print(df_clean[numeric_columns].describe())

# Verificar valores nulos tras conversión
print("\nValores nulos tras conversión de tipos:")
print(df_clean[numeric_columns].isnull().sum())

# Verificar valores negativos
for col in numeric_columns:
    if (df_clean[col] < 0).any():
        print(f"\nValores negativos encontrados en {col}:")
        print(df_clean[df_clean[col] < 0][col])
    else:
        print(f"\nNo se encontraron valores negativos en {col}.")

# 5. Verificación final
print("\nTipos de datos después de la corrección:")
print(df_clean.dtypes)
print("\nPrimeras 5 filas del DataFrame corregido:")
print(df_clean.head())

# 6. Guardar el dataset corregido
df_clean.to_csv('TelecomX_Data_corrected.csv', index=False)
print("\nDataset corregido guardado como 'TelecomX_Data_corrected.csv'.")


Valores únicos en 'Churn' antes de la corrección:
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

Valores únicos en 'Churn' después de la corrección:
Churn
No     5398
Yes    1869
Name: count, dtype: int64

Valores únicos en variables categóricas:

customer_gender:
customer_gender
Male      3675
Female    3592
Name: count, dtype: int64

customer_Partner:
customer_Partner
No     3749
Yes    3518
Name: count, dtype: int64

customer_Dependents:
customer_Dependents
No     5086
Yes    2181
Name: count, dtype: int64

phone_PhoneService:
phone_PhoneService
Yes    6560
No      707
Name: count, dtype: int64

phone_MultipleLines:
phone_MultipleLines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64

internet_InternetService:
internet_InternetService
Fiber optic    3198
DSL            2488
No             1581
Name: count, dtype: int64

internet_OnlineSecurity:
internet_OnlineSecurity
No                     3608
Yes           

## Descripción de la tarea
En esta sección, se corrigieron los valores vacíos en `Churn`, reemplazándolos por 'No'. Se aseguró que las variables numéricas (`account_Charges.Monthly`, `account_Charges.Total`, `customer_tenure`) sean del tipo correcto (`float64` o `int64`). Se verificó la coherencia de las variables categóricas, confirmando que no hay valores inesperados, y de las variables numéricas, revisando valores nulos o negativos. El dataset corregido se guardó como `TelecomX_Data_corrected.csv` para su uso en análisis posteriores.